In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-187878
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-187878


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cluster_name = "compute-cluster"
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Existing Compute using {cluster_name}")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)

Existing Compute using compute-cluster


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1, 2.0),
    "--max_iter": choice(50, 100, 150, 200)
})
# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(  script="train.py",
                        source_directory=".",
                        environment=sklearn_env,
                        compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    run_config=src,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_concurrent_runs=4,
                                    max_total_runs=20
                                    )

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(exp_run).show()
exp_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f
Web View: https://ml.azure.com/runs/HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-187878/workspaces/quick-starts-ws-187878&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-03T01:16:27.390349][API][INFO]Experiment created<END>\n""<START>[2022-03-03T01:16:28.118865][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-03T01:16:28.753793][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f
Web View: https://ml.azure.com/runs/HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-187878/workspaces/quick-starts-ws-187878&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-03T01:16:27.141436Z',
 'endTimeUtc': '2022-03-03T01:27:09.808403Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7e80a88e-d5da-468f-9930-6b74d27c4727',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': 'infinite_space_size',
  'score': '0.8971168437025797',
  'best_child_run_id': 'HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f_9',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187878.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_6bcf5c00-1610-477d-b8fd-3f0c80ec7

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = exp_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_file_names = best_run.get_file_names()
print('Metrics: ', best_run_metrics)
print("\n\nBest run file names :", best_run_file_names)
print('\n\nRun Id: {} Accuracy: {}'.format(best_run.id, best_run_metrics['Accuracy']))
model_path = "./outputs/{}_model.joblib".format(best_run.id)
final_model = best_run.register_model(model_name = 'hyperdrive_model',model_path = model_path)
final_model.download(model_path)


Metrics:  {'Regularization Strength:': 1.5403336701799286, 'Max iterations:': 100, 'Accuracy': 0.8971168437025797}


Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f_9_model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


Run Id: HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f_9 Accuracy: 0.8971168437025797


'outputs/HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f_9_model.joblib/HD_6bcf5c00-1610-477d-b8fd-3f0c80ec749f_9_model.joblib'

In [11]:
import os
print(os.listdir('outputs'))

[]


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

In [17]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
final_data = pd.concat([x, pd.DataFrame(y)])
print(list(final_data))

train_data, test_data = train_test_split(final_data, test_size=0.2)

['age', 'marital', 'default', 'housing', 'loan', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'contact_cellular', 'contact_telephone', 'education_basic.4y', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree', 'education_unknown']
['age', 'marital', 'default', 'housing', 'loan', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_servic

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    compute_target=compute_target,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_metrics = automl_run.get_metrics()
print(automl_metrics)

In [ ]:
compute_target.delete()